In [ ]:
def make_system(L_m, L_x, L_sc_up, L_sc_down, z_x, z_y, a, 
                parallel_curve,
                transverse_soi,
                mu_from_bottom_of_spin_orbit_bands,
                k_x_in_sc,
                wraparound,
                current):
    
    ######################
    ## Define templates ##
    ######################
    
    template_barrier, template_normal, template_sc_left, template_sc_right = get_templates(
            a, transverse_soi, mu_from_bottom_of_spin_orbit_bands, k_x_in_sc)

    template_interior = template_normal
    template_edge = template_barrier
    template_top_superconductor = template_sc_left
    template_bottom_superconductor = template_sc_right

    
    ###################
    ## Define shapes ##
    ###################
    
    #----------------------
    # Define middle section

    if parallel_curve:
        curve = create_parallel_sine(0, z_x, z_y)

        curve_top = create_parallel_sine(L_m//2, z_x, z_y)
        below_shape = below_curve(curve_top)

        curve_bottom = create_parallel_sine(-L_m//2, z_x, z_y)
        above_shape = above_curve(curve_bottom)
    else:
        curve = lambda x: z_y*sin(2*np.pi / z_x * x)
        below_shape = below_curve(lambda x: curve(x) + L_m//2)
        above_shape = above_curve(lambda x: curve(x) - L_m//2)
    
    right_boundary_shape = left_of_curve(lambda y: L_x)
    left_boundary_shape = right_of_curve(lambda y: 0)
    middle_shape = INTERSECTION(below_shape, above_shape, right_boundary_shape, left_boundary_shape)
    

    #------------
    # Define edge
    edge_shape = EDGE(middle_shape)
    edge_initial_site = (0,0)

    #------------------------
    # Remove edge from middle
    interior_shape = INTERIOR(middle_shape)
    interior_initial_site = (a, a)

    #--------------------------
    # Define top superconductor
    top_superconductor_border = lambda x: L_sc_up + L_m//2 + z_y
    top_superconductor_shape = INTERSECTION(DIFFERENCE(below_curve(top_superconductor_border), middle_shape),
                                            left_boundary_shape, right_boundary_shape)
    top_superconductor_initial_site = (0,L_m)

    #-----------------------------
    # Define bottom superconductor
    bottom_superconductor_border = lambda x: -L_sc_down - L_m//2 - z_y
    bottom_superconductor_shape = INTERSECTION(DIFFERENCE(above_curve(bottom_superconductor_border), middle_shape),
                                            left_boundary_shape, right_boundary_shape)
    bottom_superconductor_initial_site = (0, -L_m-a)

    #-----------
    # Define cut
    cut_curve_top = lambda x: curve(x) + a
    cut_curve_bottom = lambda x: cut_curve_top(x) - a

    top_cut_shape = EDGE(below_curve(cut_curve_top))
    bottom_cut_shape = EDGE(below_curve(cut_curve_bottom))


    ##################
    ## Build system ##
    ##################
    
    site_colors = dict()

    #---------------
    # Create builder
    if wraparound:
        syst = kwant.Builder(kwant.TranslationalSymmetry([L_x, 0]))
    else:
        syst = kwant.Builder()

    #----------
    # Fill edge
    edge_sites = syst.fill(template_edge, edge_shape, edge_initial_site)
    add_to_site_colors(site_colors, edge_sites, 'middle_barrier')
    
    #--------------
    # Fill interior
    interior_sites = syst.fill(template_interior, interior_shape, interior_initial_site)
    add_to_site_colors(site_colors, interior_sites, 'middle_interior')

    #------------------------
    # Fill top superconductor
    if L_sc_up is not 0:
        top_superconductor_sites = syst.fill(template_top_superconductor, top_superconductor_shape, top_superconductor_initial_site)
        add_to_site_colors(site_colors, top_superconductor_sites, 'top_superconductor')

    #---------------------------
    # Fill bottom superconductor
    if L_sc_down is not 0:
        bottom_superconductor_sites = syst.fill(template_bottom_superconductor, bottom_superconductor_shape, bottom_superconductor_initial_site)
        add_to_site_colors(site_colors, bottom_superconductor_sites, 'bottom_superconductor')

    
    ##########################################
    ## Add features for current calculation ##
    ##########################################

    #---------------------
    # Make cut for current
    if current:
        top_cut = list(site for site in syst.sites() if top_cut_shape(site))
        add_to_site_colors(site_colors, top_cut, 'top_cut')
        bottom_cut = list(site for site in syst.sites() if bottom_cut_shape(site))
        add_to_site_colors(site_colors, bottom_cut, 'bottom_cut')

        cuts = (top_cut, bottom_cut)

        norbs = 4
        syst = supercurrent_matsubara.add_vlead(syst, norbs, *cuts)
    
    #####################
    ## Finalize system ##
    #####################
    
    if wraparound:
        syst = kwant.wraparound.wraparound(syst)
    
    syst = syst.finalized()
    
    #---------------------
    # Get hopping
    if current:
        hopping = supercurrent_matsubara.hopping_between_cuts(syst, *cuts, electron_blocks)
        return syst, site_colors, hopping
    else:
        return syst, site_colors

In [ ]:
import kwant
from math import sin, cos
import matplotlib.pyplot as plt
import numpy as np

import scipy.constants
import cmath

import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
from sns_system import get_templates, electron_blocks
import supercurrent_matsubara
import supercurrent

In [ ]:
s1 = Shape()
s2 = Shape()

In [ ]:
from types import SimpleNamespace

In [ ]:
def slice_func(_slice):
    if _slice.start is None and _slice.stop is None:
        return lambda x: True
    elif _slice.stop is None:
        return lambda x: _slice.start <= x
    elif _slice.start is None:
        return lambda x: x < _slice.stop
    else:
        return lambda x: _slice.start <= x < _slice.stop

class Shape:
    def __init__(self, shape=None):
        if shape is None:
            self.shape = lambda site: True
        else:
            self.shape = shape
        
    def __call__(self, site):
        return self.shape(site)
    
    def __add__(self, other_shape):
        return Shape(UNION(self.shape, other_shape.shape))
    
    def __sub__(self, other_shape):
        return Shape(DIFFERENCE(self.shape, other_shape.shape))
    
    def __mul__(self, other_shape):
        return Shape(INTERSECTION(self.shape, other_shape.shape))
    
    def inverse(self):
        return Shape(NOT(self.shape))
        
    def edge(self):
        return Shape(EDGE(self.shape))
    
    def interior(self):
        return Shape(INTERIOR(self.shape))
    
    def __getitem__(self, i):
        if type(i) is tuple:
            dim = len(i)
            
            if dim == 2:
                curve_x = slice_func(i[0])
                def _shape_x(site):
                    x, y = site.pos
                    return curve_x(x)
                new_shape = self * Shape(_shape_x)
                
                curve_y = slice_func(i[1])
                def _shape_y(site):
                    x, y = site.pos
                    return curve_y(y)
                new_shape *= Shape(_shape_y)
                
            elif dim == 3:
                pass
        else:
            curve = slice_func(i)
            def _shape(site):
                x = site.pos
                return curve(x)
            new_shape = self * Shape(_shape)
        
        return new_shape
    
def NOT(shape):
    """returns not(shape)"""
    return lambda site: not shape(site)

def UNION(*shapes):
    """returns shape which is an OR function applied to shapes"""
    return lambda site: any(shape(site) for shape in shapes)

def INTERSECTION(*shapes):
    """returns shape which is an AND function applied to shapes"""
    return lambda site: all(shape(site) for shape in shapes)
    
def DIFFERENCE(shape_A, shape_B):
    """returns shape which is true when the site is in the first shape, but not in the second"""
    return lambda site: shape_A(site) and not shape_B(site)

def EDGE(shape):
    def _shape(site):
        sites = [('wsite', [-1, 0]), ('esite', [1, 0]),
                 ('nsite', [0, 1]), ('ssite', [0, -1]),
                 ('nwsite', [-1, 1]), ('nesite', [1, 1]),
                 ('swsite', [-1, -1]), ('sesite', [1, -1])]
        s = lambda x: kwant.builder.Site(site.family, site.tag + x)
        neighboring_sites = {k: s(x) for k, x in sites}
        return shape(site) and not all(map(shape, neighboring_sites.values()))
    return _shape

def INTERIOR(shape):
    return DIFFERENCE(shape, EDGE(shape))

def TRANSLATE(shape, vector):
    def _shape(site):
        translated_site  = kwant.builder.Site(site.family, site.tag - vector)
        return shape(translated_site)
    return _shape


def below_curve(curve):
    def _shape(site):
        x, y = site.pos
        return y < curve(x)
    return Shape(_shape)

def above_curve(curve):
    return Shape(below_curve(curve).inverse())

def left_of_curve(curve):
    def _shape(site):
        x, y = site.pos
        return x < curve(y)
    return _shape

def right_of_curve(curve):
    return NOT(left_of_curve(curve))

def add_to_site_colors(site_colors, marked_sites, color):
    site_colors.update({site: color for site in marked_sites})
    
def site_color_function(site_color, syst):
    return [site_color[site] for site in syst.sites]


In [ ]:
def NOT(shape):
    """returns not(shape)"""
    return lambda site: not shape(site)

def UNION(*shapes):
    """returns shape which is an OR function applied to shapes"""
    return lambda site: any(shape(site) for shape in shapes)

def INTERSECTION(*shapes):
    """returns shape which is an AND function applied to shapes"""
    return lambda site: all(shape(site) for shape in shapes)
    
def DIFFERENCE(shape_A, shape_B):
    """returns shape which is true when the site is in the first shape, but not in the second"""
    return lambda site: shape_A(site) and not shape_B(site)

def EDGE(shape):
    def _shape(site):
        sites = [('wsite', [-1, 0]), ('esite', [1, 0]),
                 ('nsite', [0, 1]), ('ssite', [0, -1]),
                 ('nwsite', [-1, 1]), ('nesite', [1, 1]),
                 ('swsite', [-1, -1]), ('sesite', [1, -1])]
        s = lambda x: kwant.builder.Site(site.family, site.tag + x)
        neighboring_sites = {k: s(x) for k, x in sites}
        return shape(site) and not all(map(shape, neighboring_sites.values()))
    return _shape

def INTERIOR(shape):
    return DIFFERENCE(shape, EDGE(shape))

def TRANSLATE(shape, vector):
    def _shape(site):
        translated_site  = kwant.builder.Site(site.family, site.tag - vector)
        return shape(translated_site)
    return _shape


def below_curve(curve):
    def _shape(site):
        x, y = site.pos
        return y < curve(x)
    return Shape(_shape)

def above_curve(curve):
    return Shape(below_curve(curve).inverse())

def left_of_curve(curve):
    def _shape(site):
        x, y = site.pos
        return x < curve(y)
    return _shape

def right_of_curve(curve):
    return NOT(left_of_curve(curve))

In [ ]:
def add_to_site_colors(site_colors, marked_sites, color):
    site_colors.update({site: color for site in marked_sites})
    
def site_color_function(site_color, syst):
    return [site_color[site] for site in syst.sites]


# Make simple builder

In [ ]:
import scipy.interpolate
from scipy.optimize import fsolve
def create_parallel_sine(distance, z_x, z_y):
    def _parallel_sine(x, distance, z_x, z_y):
        g       = lambda t: z_y * sin(2*np.pi/z_x*t)
        g_prime = lambda t: z_y * 2*np.pi/z_x*cos(2*np.pi/z_x*t)
        def _x(t):
            return t - distance*g_prime(t)/np.sqrt(1 + g_prime(t)**2) - x

        def y(t):
            return g(t) + distance/np.sqrt(1 + g_prime(t)**2)

        t = fsolve(_x, x)
        return y(t)

    xdim = np.linspace(0, z_x, 1000)
    ydim = [_parallel_sine(x, distance, z_x, z_y) for x in xdim]
    
    parallel_sine = scipy.interpolate.interp1d(xdim, ydim)
    
    return lambda x: parallel_sine(x%z_x)

In [ ]:
_curve = lambda x: 4*z_y/z_x*(x%(z_x//2)) - z_y if x%z_x < z_x//2 else -4*z_y/z_x*(x%(z_x//2)) + z_y
curve = lambda x: _curve(x+z_x/4)

In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
hv.Curve((np.linspace(0,3200,1000), [curve(x) for x in np.linspace(0,3200,1000)]))

In [ ]:
def make_system(L_m, L_x, L_sc_up, L_sc_down, z_x, z_y, a,
                sawtooth,
                parallel_curve,
                transverse_soi,
                mu_from_bottom_of_spin_orbit_bands,
                k_x_in_sc,
                wraparound,
                current,
                ns_junction):
    
    ######################
    ## Define templates ##
    ######################
    assert((sawtooth and parallel_curve) is not True)
    
    template_barrier, template_normal, template_sc_left, template_sc_right = get_templates(
        a, transverse_soi, mu_from_bottom_of_spin_orbit_bands, k_x_in_sc)

    template_interior = template_normal
    template_edge = template_barrier
    template_top_superconductor = template_sc_left
    template_bottom_superconductor = template_sc_right

    
    if parallel_curve:
        curve = create_parallel_sine(0, z_x, z_y)

        curve_top = create_parallel_sine(L_m//2, z_x, z_y)
        below_shape = below_curve(curve_top)

        curve_bottom = create_parallel_sine(-L_m//2, z_x, z_y)
        above_shape = above_curve(curve_bottom)
        
    elif sawtooth:
        _curve = lambda x: 4*z_y/z_x*(x%(z_x//2)) - z_y if x%z_x < z_x//2 else -4*z_y/z_x*(x%(z_x//2)) + z_y
        curve = lambda x: _curve(x+z_x/4)
        
        if z_y is not 0:
            theta = np.arctan(4*z_y/z_x)
            y_offset = L_m/np.cos(theta)
        else:
            y_offset = L_m
            
        below_shape = below_curve(lambda x: curve(x) + y_offset//2)
        above_shape = above_curve(lambda x: curve(x) - y_offset//2)        
        
    else:
        curve = lambda x: z_y*sin(2*np.pi / z_x * x)
        below_shape = below_curve(lambda x: curve(x) + L_m//2)
        above_shape = above_curve(lambda x: curve(x) - L_m//2)


    #--------------
    # Define middle
    middle_shape = (below_shape * above_shape)[0:L_x, :]
    
    #------------
    # Define edge
    edge_shape = middle_shape.edge()
    edge_initial_site = (0, 0)

    #------------------------
    # Remove edge from middle
    interior_shape = middle_shape.interior()
    interior_initial_site = (a, a)

    #--------------------------
    # Define top superconductor
    if sawtooth:
        top_superconductor_initial_site = (0, y_offset//2+a)
        top_superconductor_shape = middle_shape.inverse()[0:L_x, :L_sc_up + y_offset//2 + z_y]
    else:
        top_superconductor_initial_site = (z_x//4, L_m//2+z_y+a)
        top_superconductor_shape = middle_shape.inverse()[0:L_x, :L_sc_up + L_m//2 + z_y]
    
    #-----------------------------
    # Define bottom superconductor
    if sawtooth:
        bottom_superconductor_initial_site = (0, -y_offset//2-a)
        bottom_superconductor_shape = middle_shape.inverse()[0:L_x, -L_sc_down - y_offset//2 - z_y:]
    else:
        bottom_superconductor_initial_site = (z_x//4, -L_m//2-z_y-2*a)
        bottom_superconductor_shape = middle_shape.inverse()[0:L_x, -L_sc_down - L_m//2 - z_y:]

    #-----------
    # Define cut
    cut_curve_top = above_curve(lambda x: curve(x))
    cut_curve_bottom = below_curve(lambda x: curve(x))

    top_cut_shape = above_curve(curve).edge()
    bottom_cut_shape = below_curve(curve).edge()

    #-------------------
    # NS junction shapes
    if ns_junction:
        middle_shape = below_curve(curve).edge()[0:L_x, :]
        top_superconductor_lead_shape = Shape()[0:L_x, :]
        bottom_normal_lead_shape = Shape()[0:L_x, :]
        
    ############################
    ## Build junction system ##
    ############################
    site_colors = dict()
    if ns_junction:
        conservation_matrix = -supercurrent.sigz
        
        barrier_syst = kwant.Builder(conservation_law=conservation_matrix)
        top_superconductor_lead = kwant.Builder(kwant.TranslationalSymmetry((0, a)))
        bottom_normal_lead = kwant.Builder(kwant.TranslationalSymmetry((0, -a)),
                                           conservation_law=conservation_matrix)
        
        edge_sites = barrier_syst.fill(template_edge, middle_shape, (0, -a))
        add_to_site_colors(site_colors, edge_sites, 'middle_barrier')
        
        
        top_superconductor_lead.fill(template_top_superconductor, 
                                       top_superconductor_lead_shape, 
                                       (0,0))
        
        bottom_normal_lead.fill(template_normal,
                                 bottom_normal_lead_shape, 
                                 (0,0))
        
        top_superconductor_sites = barrier_syst.attach_lead(top_superconductor_lead)
        add_to_site_colors(site_colors, top_superconductor_sites, 'top_superconductor')

        
        normal_sites = barrier_syst.attach_lead(bottom_normal_lead)
        add_to_site_colors(site_colors, normal_sites, 'middle_interior')

      
        barrier_syst = barrier_syst.finalized()
        return barrier_syst, site_colors, None
    
    ##################
    ## Build system ##
    ##################        
    else:
        #---------------
        # Create builder
        if wraparound:
            syst = kwant.Builder(kwant.TranslationalSymmetry([L_x, 0]))
        else:
            syst = kwant.Builder()

        #----------
        # Fill edge
        edge_sites = syst.fill(template_edge, edge_shape, edge_initial_site)
        add_to_site_colors(site_colors, edge_sites, 'middle_barrier')

        #--------------
        # Fill interior
        interior_sites = syst.fill(template_interior, interior_shape, interior_initial_site)
        add_to_site_colors(site_colors, interior_sites, 'middle_interior')

        #------------------------
        # Fill top superconductor
        if L_sc_up is not 0:
            top_superconductor_sites = syst.fill(template_top_superconductor, top_superconductor_shape, top_superconductor_initial_site)
            add_to_site_colors(site_colors, top_superconductor_sites, 'top_superconductor')

        #---------------------------
        # Fill bottom superconductor
        if L_sc_down is not 0:
            bottom_superconductor_sites = syst.fill(template_bottom_superconductor, bottom_superconductor_shape, bottom_superconductor_initial_site)
            add_to_site_colors(site_colors, bottom_superconductor_sites, 'bottom_superconductor')


        ##########################################
        ## Add features for current calculation ##
        ##########################################

        #---------------------
        # Make cut for current
        if current:
            top_cut = list(site for site in syst.sites() if top_cut_shape(site))
            add_to_site_colors(site_colors, top_cut, 'top_cut')
            bottom_cut = list(site for site in syst.sites() if bottom_cut_shape(site))
            add_to_site_colors(site_colors, bottom_cut, 'bottom_cut')

            cuts = (top_cut, bottom_cut)

            norbs = 4
            syst = supercurrent_matsubara.add_vlead(syst, norbs, *cuts)

        #####################
        ## Finalize system ##
        #####################

        if wraparound:
            syst = kwant.wraparound.wraparound(syst)

        syst = syst.finalized()

        #---------------------
        # Get hopping
        if current:
            hopping = supercurrent_matsubara.hopping_between_cuts(syst, *cuts, electron_blocks)
            return syst, site_colors, hopping
        else:
            return syst, site_colors, None

In [ ]:
syst_pars = dict(
    L_m = 200,
    L_x = 2400,
    L_sc_up = 200,
    L_sc_down = 200,
    z_x = 800,
    z_y = 0*125,
    a   = 10,
    parallel_curve = False,
    sawtooth = True,
    transverse_soi = True,
    mu_from_bottom_of_spin_orbit_bands = True,
    k_x_in_sc = True,
    wraparound = False,
    current = False,
    ns_junction=False)

In [ ]:
%%time
syst, site_colors, hopping = make_system(**syst_pars)

In [ ]:
coloring =  {'middle_interior' : 'grey',
             'middle_barrier' : 'black',
             'bottom_superconductor' : 'gold',
             'top_superconductor' : 'gold',
             'top_cut' : 'red',
             'bottom_cut' : 'blue'}

for k,v in site_colors.items():
    if site_colors.get(k) is not None:
        site_colors[k] = coloring[v]

In [ ]:
# site_color=site_color_function(site_colors, syst)
kwant.plot(syst,site_color=site_color_function(site_colors, syst), dpi=400);

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = 1,
                 phase = np.pi,
                 T = 0.0,
                 V = 10)

params = dict(**constants,
              **params_raw)

In [ ]:
import spectrum

In [ ]:
density = kwant.operator.Density(syst)

In [ ]:
e, ev = spectrum.calc_spectrum(syst, params, k=4)

In [ ]:
ax=plt.axes()
kwant.plot(syst, site_color=site_color_function(site_colors, syst), dpi=200, ax=ax);
kwant.plotter.map(syst, density(ev[:,0]), dpi=200, ax=ax);
e

In [ ]:
def f(B):
    _params = params.copy()
    _params['B'] = B
    return spectrum.calc_lowest_state((None, _params), syst=syst)

In [ ]:
import adaptive
adaptive.notebook_extension()

In [ ]:
learner = adaptive.Learner1D(f, [0, 2])


In [ ]:
runner = adaptive.Runner(learner, ntasks=12)
runner.live_info()

In [ ]:
learner.plot()

In [ ]:
learner.plot()